In [1]:
import pandas as pd
df = pd.read_csv('test.csv')
df.__delitem__('id')
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0
1,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0
2,1807,1,2.8,0,1,0,27,0.9,186,3,4,1270,1366,2396,17,10,10,0,1,1
3,1546,0,0.5,1,18,1,25,0.5,96,8,20,295,1752,3893,10,0,7,1,1,0
4,1434,0,1.4,0,11,1,49,0.5,108,6,18,749,810,1773,15,8,7,1,0,1


In [2]:
X = df.drop(('clock_speed'), axis=1)
y = df['clock_speed']
feature_names = X.columns
print (feature_names)

Index(['battery_power', 'blue', 'dual_sim', 'fc', 'four_g', 'int_memory',
       'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram',
       'sc_h', 'sc_w', 'talk_time', 'three_g', 'touch_screen', 'wifi'],
      dtype='object')


In [3]:
print (X.shape)
print (y.shape)
N, d = X.shape

(1000, 19)
(1000,)


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 17)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print (N_train, N_test)

700 300


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

regr = RandomForestRegressor(max_depth=25, random_state=99, n_estimators=200)
regr.fit(X_train, y_train)
reg_pred = regr.predict(X_test)

mse_rf = metrics.mean_squared_error(y_test, reg_pred)
print(mse_rf)

0.8019937255417815


In [7]:
mae_rf = metrics.mean_absolute_error(y_test, reg_pred)
print(mae_rf)

0.7888357575757575


In [8]:
regr.score(X_test, y_test)

-0.12105701529296997

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

regl = LinearRegression(normalize=True)
regl.fit(X_train, y_train)
y_pred = regl.predict(X_test)

mse_l = metrics.mean_squared_error(y_test, y_pred)
print(mse_l)

0.7541041885592006


In [10]:
mae_l = metrics.mean_absolute_error(y_test, y_pred)
print(mae_l)

0.7631900135253125


In [11]:
regl.score(X_test, y_test)

-0.05411521801495933

In [12]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

regb = BaggingRegressor(DecisionTreeRegressor(max_depth=25, random_state=150), max_samples=1.0, max_features=1.0,)
regb.fit(X_train, y_train)
pred_b = regb.predict(X_test)

mse_b = metrics.mean_squared_error(y_test, pred_b)
print(mse_b)

0.8828716666666666


In [13]:
mae_b = metrics.mean_absolute_error(y_test, pred_b)
print(mae_b)

0.8237666666666666


In [14]:
regb.score(X_test, y_test)

-0.23411124551061113

In [15]:

from sklearn.neighbors import KNeighborsRegressor

regk = KNeighborsRegressor(n_neighbors = 500)
regk.fit(X_train, y_train)
pred_k = regk.predict(X_test)

mse_k = metrics.mean_squared_error(y_test, pred_k)
print(mse_k)

0.7257492489333334


In [16]:
mae_k = metrics.mean_absolute_error(y_test, pred_k)
print(mae_k)

0.7582486666666667


In [17]:
regk.score(X_test, y_test)

-0.014479616172422105

In [18]:
pers_rf = (mse_rf + mae_rf) / 2
pers_b = (mse_b + mae_b) / 2
pers_l = (mse_l + mae_l) / 2
pers_k = (mse_k + mae_k) / 2
h = [pers_k, pers_l, pers_b, pers_rf]
h = min(h)

In [19]:
if pers_rf == h:
    print ('Random Forest Regressor mse:', mse_rf, 'mae:', mae_rf)
elif pers_l == h:
    print ('Logistic Regression mse:', mse_l, 'mae:', mae_l)
elif pers_k == h:
    print ('K Neighbors Regressor mse:', mse_k, 'mae:', mae_k)
elif pers_b == h:
    print ('Bagging mse:', mse_b, 'mae', mae_b)

K Neighbors Regressor mse: 0.7257492489333334 mae: 0.7582486666666667


In [21]:
from mlxtend.regressor import StackingRegressor

stregr = StackingRegressor(regressors=[regl, regb, regk], meta_regressor=regr)
stregr.fit(X_train, y_train)
pred_s = stregr.predict(X_test)

mse_s = metrics.mean_squared_error(y_test, pred_s)
print (mse_s)

1.1619105775000003


In [22]:
mae_s = metrics.mean_absolute_error(y_test, pred_s)
print(mae_s)

0.9024650000000001


In [ ]:
stregr.score(X_test, y_test)